# Идентификация
Определение, кто изображен на фото

In [1]:
from sklearn import neighbors
from sklearn import svm
import os
import math
import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras import backend as K
K.clear_session()

In [2]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('utils/shape_predictor_5_face_landmarks.dat')
def detect_faces(img):
    dets = detector(img, 1)
    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(img, detection))
    return faces
def aligned_faces(img, faces):
    return dlib.get_face_chips(img, faces, size=224)
def aligned_face(img, face):
    return dlib.get_face_chip(img, face, size=224)

In [3]:
model = VGGFace(include_top=False, model='vgg16', input_shape=(224, 224, 3), pooling='avg')

In [4]:
X_train, y_train = [], []
for person in os.listdir('train'):
    for person_img in os.listdir("train/" + person):
        path = 'train/' + person + '/' + person_img
        print(path)
        img = load_image(path)
        faces = detect_faces(img)
        if len(faces) != 1:
            print(path, 'was skipped because of len(faces)=', len(faces))
            continue
        face_image = aligned_face(img, faces[0])
        face_image = preprocess_input(face_image.astype(float), version=1)
        encoding = model.predict(np.array([face_image]))[0]
        X_train.append(encoding)
        y_train.append(person)
X_test, y_test = [], []
for label in os.listdir('test'):
    img = load_image('test/' + label)
    X_test.append(img)
    y_test.append(label)

train/alex_lacamoire/img1.jpg
1/1 [==============================] - 4s 4s/step
train/biden/biden.jpg
1/1 [==============================] - 0s 22ms/step
train/biden/biden2.jpg
1/1 [==============================] - 0s 23ms/step
train/kit_harington/john1.jpeg
1/1 [==============================] - 0s 21ms/step
train/kit_harington/john2.jpeg
1/1 [==============================] - 0s 26ms/step
train/obama/obama.jpg
1/1 [==============================] - 0s 21ms/step
train/obama/obama2.jpg
1/1 [==============================] - 0s 25ms/step
train/rose_leslie/img1.jpg
1/1 [==============================] - 0s 21ms/step
train/rose_leslie/img2.jpg
1/1 [==============================] - 0s 23ms/step


# Нормализация и кодирование

In [6]:
from sklearn.preprocessing import Normalizer, LabelEncoder
in_encoder = Normalizer(norm='l2')
Xtrain = in_encoder.transform(X_train)

out_encoder = LabelEncoder()
ytrain = out_encoder.fit_transform(y_train)

## Извлечение эмбеддингов

In [7]:
image_id = 1
image = X_test[image_id]
faces = detect_faces(image)
_aligned_faces = aligned_faces(image, faces)
_aligned_faces = [preprocess_input(face.astype(float), version=1) for face in _aligned_faces]
embeddings = model.predict(np.array(_aligned_faces))
print('actually this is', y_test[image_id])

1/1 [==============================] - 0s 23ms/step
actually this is johnsnow_test1.jpg


In [8]:
embeddings_normalized = in_encoder.transform(embeddings)

# Метод ближайших соседей

### Обучение классификатора

In [9]:
# число соседей надо будет подобрать
knn_clf = neighbors.KNeighborsClassifier(n_neighbors=int(round(math.sqrt(len(X_train)))), algorithm='ball_tree', weights='distance')
knn_clf.fit(Xtrain, ytrain)

KNeighborsClassifier(algorithm='ball_tree', n_neighbors=3, weights='distance')

### Предсказание

In [10]:
closest_distances = knn_clf.kneighbors(embeddings_normalized, n_neighbors=1)
closest_distances

(array([[0.49085724]]), array([[3]], dtype=int64))

In [11]:
out_encoder.inverse_transform(knn_clf.predict(embeddings_normalized))

array(['kit_harington'], dtype='<U14')

In [12]:
distance_threshold = 0.7
are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(faces))]
res = [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(embeddings_normalized), faces, are_matches)]
out_encoder.inverse_transform([res[0][0]]) if res[0][0] != 'unknown' else 'unknown'

array(['kit_harington'], dtype='<U14')

# Метод опорных векторов

### Обучение классификатора

In [13]:
clf = svm.SVC(kernel='linear', probability=True)
clf.fit(Xtrain, ytrain)

SVC(kernel='linear', probability=True)

### Предсказание

In [14]:
preds = clf.predict(embeddings_normalized)

In [15]:
confidence = [proba[pred] * 100 for proba, pred in zip(clf.predict_proba(embeddings), preds)]
confidence

[1.3726527166863417e-12]

In [16]:
out_encoder.inverse_transform(preds)

array(['kit_harington'], dtype='<U14')